# Experimentación métodos iterativos

In [ ]:
import numpy as np
import subprocess as sp
import matplotlib.pyplot as plt
import json

dir_datos = "../data/sistemas"
dir_resultados = "../data/resultados"
dir_tiempos = "../data/tiempos"

## Funciones auxiliares

In [ ]:
n = 10

def correr_algoritmo(tipo, tam, metodo, iteraciones=0, tol=0):        
    proceso = sp.run(["../src/iterativo", f"{dir_datos}/{tipo}_{tam}.txt", metodo, str(iteraciones), str(tol)], capture_output=True, text=True)
    proceso.check_returncode()

    return np.array(proceso.stdout.split(" "), dtype=np.float64)

def cargar_soluciones(tipo, xs):
    for i in range(2, n + 1):
        tam = 2 ** i
        xs.append(np.genfromtxt(f"{dir_datos}/x_{tam}.txt"))

def cargar_soluciones_metodos(metodo):
    with open(f"{dir_resultados}/{metodo}.json", "r") as input:
        return json.load(input)
    
def cargar_tiempos_metodos(metodo):
    with open(f"{dir_tiempos}/{metodo}.json", "r") as input:
        return json.load(input)
    
def guardar_resultados(data, metodo):
    with open(f"{dir_resultados}/{metodo}.json", "w") as output:
        json.dump(data, output)

def guardar_tiempos(data, metodo):
    with open(f"{dir_tiempos}/{metodo}.json", "w") as output:
        json.dump(data, output)

## Correr experimentos

In [ ]:
max_iter = 450
step = 30
tolerancia = 1e-10

tipos = ["edd", "sim", "triang"]

In [ ]:
def calcular_tiempos(dic, metodo):
    for tipo in tipos:
        dic[f"{tipo}"] = []
        for i in range(2, n + 1):
            t = %timeit -o correr_algoritmo(ds, tipo, 2 ** i, metodo, max_iter, tolerancia)
            dic[f"{tipo}"].append([t.average, t.stdev])

## Soluciones de los métodos

In [ ]:
def obtener_soluciones(dic, metodo):
    for tipo in tipos:
        dic[f"{tipo}"] = []
        for i in range(2, n + 1):
            if metodo != "LU":
                for j in range(1, max_iter + 1, step):
                    res = correr_algoritmo(tipo, 2 ** i, metodo, j, tolerancia)
                    dic[f"{tipo}"].append(res.tolist())
            else:
                res = correr_algoritmo(tipo, 2 ** i, metodo)
                dic[f"{tipo}"].append(res.tolist())

### Jacobi

In [ ]:
resultados_jacobi = {}
obtener_soluciones(resultados_jacobi, "J")
guardar_resultados(resultados_jacobi, "jacobi")

### Jacobi Sumatoria

In [ ]:
resultados_jacobi_sum = {}
obtener_soluciones(resultados_jacobi_sum, "JS")
guardar_resultados(resultados_jacobi_sum, "jacobi_sum")

### Gauss Seidel

In [ ]:
resultados_gs = {}
obtener_soluciones(resultados_gs, "GS")
guardar_resultados(resultados_gs, "gauss_seidel")

### Gauss Seidel Sumatoria

In [ ]:
resultados_gs_sum = {}
obtener_soluciones(resultados_gs_sum, "GSS")
guardar_resultados(resultados_gs_sum, "gauss_seidel_sum")

### LU

In [ ]:
resultados_lu = {}
obtener_soluciones(resultados_lu, "LU")
guardar_resultados(resultados_lu, "lu")

## Cargar vectores solución

In [ ]:
xs = []

cargar_soluciones("", xs)


## Cargar soluciones métodos

In [ ]:
sol_j = cargar_soluciones_metodos("jacobi")
sol_js = cargar_soluciones_metodos("jacobi_sum")
sol_gs = cargar_soluciones_metodos("gauss_seidel")
sol_gss = cargar_soluciones_metodos("gauss_seidel_sum")
sol_lu = cargar_soluciones_metodos("lu")


## Tiempo de cómputo para los métodos

In [ ]:
tiempos_jacobi = {}
calcular_tiempos(tiempos_jacobi, "J")
guardar_resultados(tiempos_jacobi, "tiempos_jacobi")

In [ ]:
tiempos_jacobi_sum = {}
calcular_tiempos(tiempos_jacobi_sum, "JS")
guardar_resultados(tiempos_jacobi_sum, "tiempos_jacobi_sum")

In [ ]:
tiempos_gs = {}
calcular_tiempos(tiempos_gs, "GS")
guardar_resultados(tiempos_gs, "tiempos_gauss_seidel")

In [ ]:
tiempos_gss = {}
calcular_tiempos(tiempos_gss, "GSS")
guardar_resultados(tiempos_gss, "tiempos_gauss_seidel_sum")

In [ ]:
tiempos_lu = {}
calcular_tiempos(tiempos_lu, "LU")
guardar_resultados(tiempos_gs, "tiempos_lu")

## Generación de Gráficos

### Error

In [ ]:
jacobi_separados = {}
gs_separados = {}
jacobi_sum_separados = {}
gs_sum_separados = {}

# separamos los resultados por tamaño de matriz
for tipo in tipos:
    jacobi_separados[f"{tipo}"] = {}
    gs_separados[f"{tipo}"]  = {}
    jacobi_sum_separados[f"{tipo}"]  = {}
    gs_sum_separados[f"{tipo}"]  = {}
    for i in range(2, n+1):
        jacobi_separados[f"{tipo}"][f"{2**i}"] = []
        gs_separados[f"{tipo}"][f"{2**i}"]  = []
        jacobi_sum_separados[f"{tipo}"][f"{2**i}"]  = []
        gs_sum_separados[f"{tipo}"][f"{2**i}"]  = []
        for sol in sol_j[f"{tipo}"]:
            if len(sol) == 2**i:
                jacobi_separados[f"{tipo}"][f"{2**i}"].append(sol)
        for sol in sol_gs[f"{tipo}"]:
            if len(sol) == 2**i:
                gs_separados[f"{tipo}"][f"{2**i}"].append(sol)
        for sol in sol_js[f"{tipo}"]:
            if len(sol) == 2**i:
                jacobi_sum_separados[f"{tipo}"][f"{2**i}"].append(sol)
        for sol in sol_gss[f"{tipo}"]:
            if len(sol) == 2**i:
                gs_sum_separados[f"{tipo}"][f"{2**i}"].append(sol)
                
errores_jacobi = {}
errores_gs = {}
errores_jacobi_sum = {}
errores_gs_sum = {}

for tipo in tipos:
    errores_jacobi[f"{tipo}"] = {}
    errores_gs[f"{tipo}"] = {}
    errores_jacobi_sum[f"{tipo}"] = {}
    errores_gs_sum[f"{tipo}"] = {}
    for sol in xs:
        errores_jacobi[f"{tipo}"][f"{len(sol)}"] = []
        errores_gs[f"{tipo}"][f"{len(sol)}"] = []
        errores_jacobi_sum[f"{tipo}"][f"{len(sol)}"] = []
        errores_gs_sum[f"{tipo}"][f"{len(sol)}"] = []

        for x in jacobi_separados[f"{tipo}"][f"{len(sol)}"]:
            errores_jacobi[f"{tipo}"][f"{len(sol)}"].append(np.linalg.norm(x-sol))
        for x in gs_separados[f"{tipo}"][f"{len(sol)}"]:
            errores_gs[f"{tipo}"][f"{len(sol)}"].append(np.linalg.norm(x-sol))
        for x in jacobi_sum_separados[f"{tipo}"][f"{len(sol)}"]:
            errores_jacobi_sum[f"{tipo}"][f"{len(sol)}"].append(np.linalg.norm(x-sol))
        for x in gs_sum_separados[f"{tipo}"][f"{len(sol)}"]:
            errores_gs_sum[f"{tipo}"][f"{len(sol)}"].append(np.linalg.norm(x-sol))

In [ ]:
tam = 128
tipo = "edd"
errs = errores_jacobi[f"{tipo}"][f"{tam}"]

its = range(1,max_iter + 1,step)

plt.plot(its, errs)
plt.xlabel("N iteraciones")
plt.ylabel("Error (norma 2)")
plt.title(f"Error met. Jacobi, para matriz e.d.d. de tamaño {tam}x{tam}",fontsize = 9)
plt.savefig(f"../plots/err_jacobi_{tipo}_{tam}")

In [ ]:
tam = 128
tipo = "edd"
errs = errores_gs[f"{tipo}"][f"{tam}"]

its = range(1,max_iter + 1,step)

plt.plot(its, errs)
plt.xlabel("N iteraciones")
plt.ylabel("Error (norma 2)")
plt.title(f"Error met. G-S, para matriz e.d.d. de tamaño {tam}x{tam}",fontsize = 9)
plt.savefig(f"../plots/err_gs_{tipo}_{tam}")

### Tiempo de cómputo

In [ ]:
tiempos_j = cargar_tiempos_metodos("jacobi")
tiempos_js = cargar_tiempos_metodos("jacobi_sum")
tiempos_gs = cargar_tiempos_metodos("gauss_seidel")
tiempos_gss = cargar_tiempos_metodos("gauss_seidel_sum")
tiempos_lu = cargar_tiempos_metodos("lu")

In [ ]:
tam_matriz = [2 ** i for i in range(2, n + 1)]
def graficar_tiempos(dic, tipo, nombre_metodo):
    _, ax = plt.subplots()
    for ds in datasets: 
        t = [tiempo[0] * 1000 for tiempo in dic[ds][tipo]]
        ax.plot(tam_matriz, t)

    ax.set_ylabel("Tiempo (ms)")
    ax.set_xlabel("Tamaño matriz")
    ax.set_title(f"{nombre_metodo} dataset {tipo}")
    ax.legend(datasets)
    
    plt.show()

def graficar_desvio_tiempo(dic, tipo, nombre_metodo):
    _, ax = plt.subplots()
    datos = [[dato[0] * 1000, dato[1] * 1000] for dato in dic["chico"][tipo]]
    ax.boxplot(datos)
    
    ax.set_xticklabels(tam_matriz)
    ax.set_ylabel("Tiempo (ms)")
    ax.set_xlabel("Tamaño matriz")
    ax.set_title(f"{nombre_metodo} dataset {tipo}")

    plt.show()

In [ ]:
graficar_tiempos(tiempos_j, "edd", "Jacobi")
graficar_tiempos(tiempos_js, "edd", "Jacobi Sumatoria")
graficar_tiempos(tiempos_gs, "edd", "Gauss-Seidel")
graficar_tiempos(tiempos_gss, "edd", "Gauss-Seidel Sumatoria")

In [ ]:
graficar_desvio_tiempo(tiempos_j, "edd", "Jacobi")

### LU

### LU vs Métodos Iterativos

- Error de aproximacion entre valor actual y final (por cantidad de iteraciones) para matrices de diferentes tamaños
- Tiempo final de computo por iteraciones y por tamaño de matrices
- LU tiempo de computo por tamaño de matriz. Error numerico (comparar con x real).
- Comparar LU con todos los metodos iterativos.